# 4.2 Numerical Integration and Differentiating

**Today's class:**

* review workflow personal mp248 repo
* The `Index.md` file
* Langtangen Book
* Quiz


* Introduction to functions
    - def
    - lambda
* Numerical derivatives
    - Difference equations
    - Derivatives
    - Errors: accuracy vs. precision
    - Convergence
    - Higher-order derivative

In [ ]:
%pylab ipympl

## Functions
There are two common ways to express functions. A function in computing is a bit different from a [mathematical function](https://en.wikipedia.org/wiki/Function_(mathematics)). It has a number of inputs and a number of outputs, and typically organizes a particular _function_ in a code object.

Let's express this mathematical function in Python:
$$
f(x) = ax^3 + bx^2 + cx + d
$$
for $a = -1$, $b = 1$, $c = 1$, $d = 1$.

### Def
```Python
def fpoly(x,a=-1.,b=1.,c=1.,d=1.):
    '''Return 3rd-order polynomial
    '''
    ff = a*x**3 + b*x**2 + c*x + d
    return ff
```
Arguments with `=` are optional arguments, and the values given in the function definition are the default values, used if the parameters are not specified in the function call. Conversely, arguments without `=` are required.

Functions can have more than one required argument. $$ z(x,y) = \sin(x y)\cos(y) $$ would be
```Python
    def ztrig(xx,yy):
        '''2-variable trig function
        '''
        return sin(xx*yy)*cos(yy)
```

In [13]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


In [6]:
def fpoly(x,a=-1.,b=1.,c=1.,d=1.):
    '''Return 3rd-order polynomial
    
    Parameters
    ----------
    x :: float 
      input value
    
    a,b,c,d :: float (optional)
      coefficients
    
    Returns:
    --------
    f :: float
      function value
    '''
    ff = a*x**3 + b*x**2 + c*x + d
    return ff


In [7]:
fpoly?

Signature: fpoly(x, a=-1.0, b=1.0, c=1.0, d=1.0)
Docstring:
Return 3rd-order polynomial

Parameters
----------
x :: float 
  input value

a,b,c,d :: float (optional)
  coefficients

Returns:
--------
f :: float
  function value
File:      ~/mp248-course-notes/<ipython-input-6-a0ec0bc2d51a>
Type:      function


### Lambda
```Python
a = -1.; b = 1.; c = 1.; d = 1.
fpoly = lambda x: a*x**3 + b*x**2 + c*x + d
```


In both cases the function is used as 
```Python
x = 2
fpoly(x)
```

**Example:** Make a line plot for the polynomial given above for $x \in [-2,2]$.

In [9]:
def fpoly(x,a=-1.,b=1.,c=1.,d=1.):
    '''Return 3rd-order polynomial
    '''
    ff = a*x**3 + b*x**2 + c*x + d
    return ff

In [10]:
def fpoly_deriv(x,a=-1.,b=1.,c=1.,d=1.):
    '''Return derivative of 3rd-order polynomial
    '''
    ff = 3.*a*x**2 + 2*b*x + c
    return ff

In [11]:
fpoly_deriv(1)

0.0

In [18]:
x = linspace(-2,2,101)
ifig=11;close(ifig);figure(ifig)
plot(x,fpoly(x,a=2,b=10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def ztrig(xx,yy):
    '''2-variable trig function
    '''
    return sin(xx*yy)*cos(yy)

In [16]:
x = 1.2; y = -0.3
ztrig(x,y)

-0.3365404292264392

## Numerical derivatives 

In this section we will learn how we can calculate a derivative numerically. The fundamental idea involves replacing differentials with differences. An important aspect is to think about the types of errors that emerge when doing that, especially we have to introduce the important concepts of rounding error and truncation errors.

**Literature:** [Numerical Recipies](http://voyager.library.uvic.ca/vwebv/holdingsInfo?bibId=687673), Ch. 5.7

### Difference equations
Function
$$ y = f(x) $$ for example $$y = x^2$$ for $x \in [1,5]$.

In [19]:
x=linspace(5,25,25)

In [27]:
figure(1)
xl =  4; xr = 22
f = lambda x: x**2    # function
x0 = 10               # take derivative at x0
ff = lambda x: 2*x0*x + f(x0) -2*x0**2   # derivative
close(1);figure(1)
plot(x,f(x)); xlabel('x'), ylabel('y = f(x)'); ylim(0,500)
if True:
    plot(x0,f(x0),'o')
    plot(x[4:22],ff(x[4:22]),'-.',lw=2)
if True:
    h = 10
    x1=10.;x2=x1 + h
    plot([x1,x2],[f(x1),f(x1)],'k--x',lw=1)
    plot([x2,x2],[f(x1),f(x2)],'k--x',lw=1)
    text(0.5*(x2+x1),1.1*ff(x1),'$ \Delta x$')
    text(1.02*x2,0.5*(ff(x1)+ff(x2)),'$ \Delta y$')
    text(x1,0.5*f(x1),'$x_1$')
    text(x2,0.5*f(x1),'$x_2$')
    text(1.03*x2,1.*f(x1),'$y_1  = f(x_1)$')
    text(0.8*x2,1.0*f(x2),'$y_2 = f(x_2)$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Slope
$$
\frac{dy}{dx}  \approx \frac{\Delta y}{\Delta x} = 
\frac{y_2 - y_1}{x_2 - x_1}
$$

### Derivative 
The derivative $\frac{df}{dx}$ of a function $y=f(x)$ can be approximated by the difference equation 
$$ f'(x) \approx \frac{f(x+h) -f(x)}{h}.$$

Why? Rearrange the Taylor expansion of $f(x)$
$$
f(x+h) = f(x) + hf^\prime(x) + \frac{1}{2}h^2f^{\prime\prime}(x)
 + \frac{1}{6}h^3f^{\prime\prime\prime}(x) + \dots
$$
to solve for $f^\prime(x)$ and discard order two and higher terms
$$
\frac{1}{2}h^2f^{\prime\prime}(x)
 + \frac{1}{6}h^3f^{\prime\prime\prime}(x) + \dots
$$

Thus:
$$
 f'(x)=\frac{f(x+h) -f(x)}{h} +\mathcal{O}(2)
$$

In [28]:
def deriv1(f,x,h):
    '''Derivative 1'''
    dfdx = (f(x+h) - f(x)) / h
    return dfdx

In [29]:
f(2)

4

In [38]:
deriv1(f,10,1.e-10)-20.

7.270908099599183e-05

Note that the answer is not exactly `2.00000000` as we know the answer to be from the analytical approach. But smaller values of `h` give a better answer? _Right?_ well, let's see .... let's analyse the error in a bit more detail. The higher order terms that we discarded represent the truncation error. They represent the difference between the solution of the difference equation and the exact mathematical equation, no matter how _precisely_ the difference equation is solved. The truncation error limits the _accuracy_ of the method. Note the difference between _precision_ and _accuracy_ introducecd here!

We can improve the accuracy by taking smaller $h$ for calculating the derivative. Can't we make the answer then arbitrarily exact? Let's try ...

### Errors
There are two basic sources of error! They have to do with accuracy and with precision.

**Accuracy:** How well does the discretized equation represent the original mathematical equation?
This is the truncation error. 

**Precision:** How well is the solution satisfying the discretized equation? This is limited by the roundoff error.

### Convergence test
In order to determine the accuracy of our solution scheme we study the behaviour of a particular solution scheme under grid (time and/or space) refinement. In our case this means we do a series of runs with decreasing `h`.

In [39]:
h = 0.001
deriv1(f,1,h)

2.0009999999996975

In [40]:
h_pow = range(0,-14,-1)

In [41]:
h_pow

range(0, -14, -1)

In [42]:
h = 10**array(h_pow, dtype=float)

In [43]:
h

array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07,
       1.e-08, 1.e-09, 1.e-10, 1.e-11, 1.e-12, 1.e-13])

In [45]:
abs(deriv1(f,1.,1.e-8)-2.0)

1.21549419418443e-08

In [46]:
log10(abs(deriv1(f,1.,1.e-8)-2.0))

-7.915247111225982

In [47]:
log10(abs(deriv1(f,1.,h)-2.0))

array([ 0.        , -1.        , -2.        , -3.        , -4.        ,
       -4.9999994 , -6.00003285, -6.99530123, -7.91524711, -6.78125254,
       -6.78125254, -6.78125254, -3.7500654 , -2.79627223])

In [48]:
close(2);figure(2)
fresids = log10(abs(deriv1(f,1.,h)-2.0))
plot(h_pow, fresids,'o')
xlim(0,-14), ylabel('log10 (df/dx - 2.0)'), xlabel('log h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((0, -14), Text(0, 0.5, 'log10 (df/dx - 2.0)'), Text(0.5, 0, 'log h'))

#### Bottom line
You can reduce the truncation error by taking a smaller interval `h`. However, as you try minimizing the truncation error you will see that there is a limit. At too small `h` the `deriv1` function will try to perform the difference `(f(x+h) - f(x))` with too few or no significant digits and the roundoff error will become larger, and eventually dominate the result. 


**Example:** Calculate $\frac{d fpoly(1)}{dx}$

In [49]:
deriv1(fpoly,-0.5,0.1)

-0.5099999999999993

### Higher-order derivative

(1)   $ f(x+h) = f(x) + hf^\prime(x) + \frac{1}{2}h^2f^{\prime\prime}(x)
 + \frac{1}{6}h^3f^{\prime\prime\prime}(x) + \dots$


(2) $
f(x-h) = f(x) - hf^\prime(x) + \frac{1}{2}h^2f^{\prime\prime}(x)
 - \frac{1}{6}h^3f^{\prime\prime\prime}(x) + \dots
$


Eqn (3): (2) - (1) 
$$f(x+h) - f(x-h) =  2hf^\prime(x) 
 + \frac{2}{3}h^3f^{\prime\prime\prime}(x) + \dots$$
$$ f^\prime(x)  =  \frac{f(x+h) - f(x-h)}{2h} + \mathcal{O}(3)$$